# Programe sua GPU com OpenMP

Autores:
_Hermes Senger_ e
_Jaime Freire de Souza_

Data de criação:     16/04/2022   
Última modificação:     

## Configuração do ambiente

Precisaremos de um compilador capaz de gerar código executável para GPUs.


In [1]:
%%shell
ln -sfnv /usr/local/cuda-11/ /usr/local/cuda
pip install -q matplotlib numpy
wget https://openmp-course.s3.amazonaws.com/llvm.tar.gz
tar -xzvf llvm.tar.gz >/dev/null 2>&1
echo "  ------------  Terminou a instalação! Pode continuar  ------------------"

'/usr/local/cuda' -> '/usr/local/cuda-11/'
--2023-07-17 01:17:01--  https://openmp-course.s3.amazonaws.com/llvm.tar.gz
Resolving openmp-course.s3.amazonaws.com (openmp-course.s3.amazonaws.com)... 52.216.42.33, 3.5.6.124, 3.5.27.170, ...
Connecting to openmp-course.s3.amazonaws.com (openmp-course.s3.amazonaws.com)|52.216.42.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810538565 (773M) [application/x-gzip]
Saving to: ‘llvm.tar.gz’

llvm.tar.gz         100%[===================>] 772.99M  33.5MB/s    in 24s     

2023-07-17 01:17:25 (32.6 MB/s) - ‘llvm.tar.gz’ saved [810538565/810538565]

  ------------  Terminou a instalação! Pode continuar  ------------------



Também é preciso informar a localização de bibliotecas e executáveis.


In [2]:
import os

os.environ['LLVM_PATH'] = '/content/llvm'
os.environ['PATH'] = os.environ['LLVM_PATH'] + '/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = os.environ['LLVM_PATH'] + '/lib:' + os.environ['LD_LIBRARY_PATH']
os.environ['TSAN_OPTIONS'] = 'ignore_noninstrumented_modules=1'


Agora poderemos testar se nosso ambiente de programação está funcionando como esperado
.

In [3]:
%%writefile test.c

#include <omp.h>
#include <stdio.h>

int main() {
  int num_devices = omp_get_num_devices();
  printf("Temos %d devices alocados\n", num_devices);
}

Writing test.c



A seguir, vamos compilar e executar o programa criado. Usaremos sempre o compilador __clang__, que utiliza o backend __llvm__, que gera código executável para GPUs de diferentes tipos e modelos.


In [4]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda -Xopenmp-target -march=sm_75 test.c -o teste

!./teste

clang-13: warning: Unknown CUDA version. cuda.h: CUDA_VERSION=11080. Assuming the latest supported version 10.1 [-Wunknown-cuda-version]
Temos 1 devices alocados


Vamos verificar o modelo de GPU alocada.

In [5]:
!nvidia-smi
!lscpu

Mon Jul 17 01:19:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Exempo 1: Calculo de Pi - serial na CPU

O programa a seguir calcula o valos re Pi pelo método de integração numérica. Esta primeira versão trabalha de forma serial, somente na CPU. Nas próximas versões, nós faremos melhorias nesse programa para acelerar o seu desempenho.


In [6]:
%%writefile ex1-pi_serial.c
// The OpenMP Common Core - pg. 58
// https://github.com/tgmattso/OmpCommonCore/blob/master/Book/C/Fig_4.5_poprogseq.c
#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();

   for (i = 0; i < num_steps; i++){
      x = (i + 0.5) * step;
      sum += 4.0 / (1.0 + x * x);
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}

Writing ex1-pi_serial.c


In [7]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda -Xopenmp-target -march=sm_75 ex1-pi_serial.c -o ex1-pi_serial

!./ex1-pi_serial

clang-13: warning: Unknown CUDA version. cuda.h: CUDA_VERSION=11080. Assuming the latest supported version 10.1 [-Wunknown-cuda-version]
pi = 3.141593, 100000000 steps, 1.417067 secs
 

## Exemplo 2: Pi paralelo na CPU - Pi-V1.0.c

O próximo exemplo mostra uma forma de paralelizar o algoritmo para execução em CPU, utilizando os seguintes recursos do OpenMP:

* Regiões paralelas
* Loops paralelos
* Redução

In [8]:
%%writefile Pi-V1.0.c

#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
   #pragma omp parallel
   {
     double x; /* cada thread terá sua variavel x local */
     #pragma omp for reduction(+:sum)
       for (i = 0; i < num_steps; i++){
          x = (i + 0.5) * step;
          sum += 4.0 / (1.0 + x * x);
       }
   }
   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("   pi = \%20.15lf, \%ld steps, \%lf secs\n", pi,
          num_steps, run_time);
}

Writing Pi-V1.0.c


Os compiladores implementam diversas otimizações durante a geração de código. Veremos a seguir, como essas otimizações podem acelerar nosso código.
Para mais detalhes sobre os flags, veja aqui:
https://clang.llvm.org/docs/CommandGuide/clang.html


In [9]:
#%%shell

#!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda Pi-V1.0.c -o Pi-V1.0
!clang -fopenmp Pi-V1.0.c -o Pi-V1.0
!./Pi-V1.0
!clang -O1 -fopenmp Pi-V1.0.c -o Pi-V1.0
!./Pi-V1.0
!clang -O2 -fopenmp Pi-V1.0.c -o Pi-V1.0
!./Pi-V1.0
!clang -O3 -fopenmp Pi-V1.0.c -o Pi-V1.0
!./Pi-V1.0
!lscpu

   pi =    3.141592653589910, 100000000 steps, 0.712020 secs
   pi =    3.141592653589910, 100000000 steps, 0.155057 secs
   pi =    3.141592653589910, 100000000 steps, 0.145885 secs
   pi =    3.141592653589910, 100000000 steps, 0.141715 secs
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:                        3
CPU MHz:                         2000.138
BogoMIPS:                        4000.27
Hypervisor vendor:               KVM
Virtua

# Exercício 1 - Soma de vetores

O exercício a seguir utiliza a cláusula __map__ para fazer a movimentação correta dos dados entre o host e o device. Sigas os passos:
1. Paralelize a soma de vetores com a diretiva `#pragma omp target` para executar na GPU
2. Paralelize o loop da inicialização na CPU com `#pragma omp parallel for`
3. Paralelizar o loop de teste na CPU. \\
Obs.: Você pode utilizar redução para totalizar a contagem de erros:
`#pragma omp parallel for reduction(+:err)`
4. O programa está disponível aqui, caso precise restaurá-lo:
https://github.com/UoB-HPC/openmp-tutorial/blob/master/vadd.c



In [10]:
%%writefile soma-vetores.c

// Copie aqui o código ...

#include <stdio.h>
#include <omp.h>
#define N 100000
#define TOL  0.0000001
//
//  This is a simple program to add two vectors
//  and verify the results.
//
//  History: Written by Tim Mattson, November 2017
//
int main()
{

    float a[N], b[N], c[N], res[N];
    int err=0;

   // fill the arrays
   for (int i=0; i<N; i++){
      a[i] = (float)i;
      b[i] = 2.0*(float)i;
      c[i] = 0.0;
      res[i] = i + 2*i;
   }

   // add two vectors
   for (int i=0; i<N; i++){
      c[i] = a[i] + b[i];
   }

   // test results
   for(int i=0;i<N;i++){
      float val = c[i] - res[i];
      val = val*val;
      if(val>TOL) err++;
   }

   printf(" Os vetores foram somados com %d erros!\n",err);
   return 0;
}

Writing soma-vetores.c


In [ ]:
#%%shell

#!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda Pi-V1.0.c -o Pi-V1.0
!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda -Xopenmp-target -march=sm_75 soma-vetores.c -o soma-vetores
!./soma-vetores



## Exercício 2: Movimentação explícita de dados - Soma de vetores na GPU

Agora vamos alocar os vetores no heap em vez do stack:
* O programa abaixo trocou `double a[N]`
* por `*a = malloc(sizeof(double) * N)`
* Use a diretiva target para descarregar a execução na GPU
`#pragma omp targtet`
* Copie os dados dos arrays no heap para/da GPU com as cláusulas map
`map(tofrom:… ), map(to:…), map(from:…)`

__Obs.:__ O código base da próxima célula foi retirado daqui, caso precise restaurá-lo: https://github.com/UoB-HPC/openmp-tutorial/blob/master/vadd_heap.c  



In [12]:
%%writefile vadd_heap.c

#include <stdio.h>
#include <omp.h>
#define N 100000
#define TOL  0.0000001
//
//  This is a simple program to add two vectors
//  and verify the results.
//
//  History: Written by Tim Mattson, November 2017
//
int main()
{

    float *a   = malloc(sizeof(float) * N);
    float *b   = malloc(sizeof(float) * N);
    float *c   = malloc(sizeof(float) * N);
    float *res = malloc(sizeof(float) * N);
    int err=0;

   // fill the arrays
   for (int i=0; i<N; i++){
      a[i] = (float)i;
      b[i] = 2.0*(float)i;
      c[i] = 0.0;
      res[i] = i + 2*i;
   }

   // add two vectors
   for (int i=0; i<N; i++){
      c[i] = a[i] + b[i];
   }

   // test results
   for(int i=0;i<N;i++){
      float val = c[i] - res[i];
      val = val*val;
      if(val>TOL) err++;
   }
   printf(" vectors added with %d errors\n",err);

   free(a);
   free(b);
   free(c);
   free(res);

   return 0;
}


Writing vadd_heap.c


In [13]:
#%%shell

#!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda Pi-V1.0.c -o Pi-V1.0
!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda -Xopenmp-target -march=sm_75 vadd_heap.c -o vadd_heap
!./vadd_heap


clang-13: warning: Unknown CUDA version. cuda.h: CUDA_VERSION=11080. Assuming the latest supported version 10.1 [-Wunknown-cuda-version]
 vectors added with 0 errors


# Exemplo 3 - Pi  V2.0 - threads na GPU

A seguir, vamos utilizar a GPU para tentar acelerar nosso programa, utilizando:
* Construção __target__ :

  `#pragma omp target`    
  `#pragma omp parallel for`   
  `     for (i=0;i<N;i++) ...`

  Modifique o programa abaixo, introduzindo a melhoria sugerida:

In [14]:
%%writefile Pi-par-V2.c

#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();

   // Criar uma região paralela no device

   {
     // Fazer o worksharing

     for (i = 0; i < num_steps; i++){
       x = (i + 0.5) * step;
       sum += 4.0 / (1.0 + x * x);
      }
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}

Writing Pi-par-V2.c


In [15]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda -Xopenmp-target -march=sm_75 Pi-par-V2.c -o Pi-par-V2

!./Pi-par-V2

clang-13: warning: Unknown CUDA version. cuda.h: CUDA_VERSION=11080. Assuming the latest supported version 10.1 [-Wunknown-cuda-version]
pi = 3.141593, 100000000 steps, 1.369899 secs
 


__Pergunta:__ O que aconteceu? Compare o tempo de execução com o da CPU.

__Resposta__:  Nesta versão, somente __um time de threads__ foi criado, para executar blocos de iterações do loop. Os threads desse time executarão de forma paralela, mas ocupando apenas um __compute unit__ apenas, o resultado não será muito bom.  

# Exemplo 4 - Pi V3.0 - múltiplos times

A seguir, vamos utilizar a GPU para tentar acelerar nosso programa, utilizando:
* Construções __target__, __teams__ e __distribute__ :

  `#pragma omp target`    
  `#pragma omp teams`    
  `#pragma omp distribute`    
  `     for (i=0;i<N;i++) ...`


  Modifique o programa abaixo, introduzindo a melhoria sugerida.

  __Obs:__
  Para este exercício, não utilize a construção  `#pragma omp parallel for`.


In [16]:
%%writefile Pi-par-V3.c

#include <stdio.h>
#include <omp.h>
static long num_steps = 1000000; //100000000
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
   #pragma omp target map(sum)
   #pragma omp teams reduction(+: sum)
    {
     double x;
     #pragma omp distribute
     for (i = 0; i < num_steps; i++){
       x = (i + 0.5) * step;
       sum += 4.0 / (1.0 + x * x);
     }
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}

Writing Pi-par-V3.c


In [17]:
#%%shell

#!clang -O3 -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda -Xopenmp-target -march=sm_37 pi-par-V3.c -o pi-par-V3
!clang -fopenmp -fopenmp-targets=nvptx64 -Xopenmp-target -march=sm_75 Pi-par-V3.c -o Pi-par-V3

!./Pi-par-V3

clang-13: warning: Unknown CUDA version. cuda.h: CUDA_VERSION=11080. Assuming the latest supported version 10.1 [-Wunknown-cuda-version]
pi = 3.141593, 1000000 steps, 1.593711 secs
 

Compare o tempo de execução com o anterior e explique por que foi mais lento que o anterior.

__Resposta__: Neste exemplo, foram criados 128 teams (depende do compilador/hardware) com apenas 1 thread (chamado de thread inicial) para cada team.

# Exemplo 5 - Pi V4.0 - times+threads+SIMD

A seguir, vamos utilizar utilizar paralelismo em 3 níveis: times, threads nos times, e SIMD nos threads para acelerar nosso programa:

  __`#pragma omp target`__    
  __`#pragma omp teams distribute`__  
    `for (i=0;i<N;i++) ...`    
  __`#pragma omp parallel for simd`__  
    `for (i=0;i<M;i++) ...`
     


In [ ]:
%%writefile Pi-par-V4.c

#define MIN(x, y) (((x) < (y)) ? (x) : (y))
#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
   #pragma omp target map(sum)
   #pragma omp teams reduction(+:sum)
   {
     int block_size = num_steps/omp_get_num_teams();
//     #pragma omp distribute dist_sched(static, 1)
     #pragma omp distribute
     for (int ii = 0; ii < num_steps; ii += block_size){
         #pragma omp parallel for simd reduction(+: sum)
//         #pragma omp parallel for reduction(+: sum)
         for (int i = ii; i < MIN(ii+block_size, num_steps); i++) {
             x = (i + 0.5) * step;
             sum += 4.0 / (1.0 + x * x);
         }
     }
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}

Writing Pi-par-V4.c


In [ ]:
#%%shell

#!clang -O3 -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda -Xopenmp-target -march=sm_37 pi-par-V3.c -o pi-par-V3
!clang  -fopenmp -fopenmp-targets=nvptx64 -Xopenmp-target -march=sm_75 Pi-par-V4.c -o Pi-par-V4

!./Pi-par-V4

clang-13: warning: Unknown CUDA version. cuda.h: CUDA_VERSION=11080. Assuming the latest supported version 10.1 [-Wunknown-cuda-version]
pi = 3.141593, 100000000 steps, 0.336792 secs
 

# Extra 1- Pi V5.0

A seguir, vamos utilizar a GPU para tentar acelerar nosso programa, utilizando:
* Construções __teams distribute__ , __parallel for simd__:

  `#pragma omp target`    
  `#pragma omp teams distribute`   
  `for (i=0;i<N;i++) ...`   
  `#pragma omp parallel for`   
    `for (i=0;i<block_sizeN;i++) ...`
     

Modifique o programa abaixo, introduzindo a melhoria sugerida.

...


In [ ]:
%%writefile Pi-par-V5.c

#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
   #pragma omp target map(sum)
   {
     #pragma omp teams distribute parallel for reduction(+:sum) private(x)
     for (i = 0; i < num_steps; i++){
       x = (i + 0.5) * step;
       sum += 4.0 / (1.0 + x * x);
     }
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}


Writing Pi-par-V5.c


In [ ]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64 -Xopenmp-target -march=sm_75 Pi-par-V5.c -o Pi-par-V5

!./Pi-par-V5

clang-13: warning: Unknown CUDA version. cuda.h: CUDA_VERSION=11080. Assuming the latest supported version 10.1 [-Wunknown-cuda-version]
pi = 3.141593, 100000000 steps, 1.741119 secs
 

# Extra 2 - Pi V6.0

Juntando tudo isso

In [ ]:
%%writefile Pi-par-V6.c
#define MIN(x, y) (((x) < (y)) ? (x) : (y))
#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;
   start_time = omp_get_wtime();
   #pragma omp target map(sum)
//   #pragma omp teams distribute parallel for simd reduction(+: sum) private(x)
   #pragma omp teams distribute parallel for reduction(+: sum) private(x)
   for (i=0; i< num_steps; i++){
             x = (i + 0.5) * step;
             sum += 4.0 / (1.0 + x * x);
   }
   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}

Writing Pi-par-V6.c


In [ ]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64 -Xopenmp-target -march=sm_75 Pi-par-V6.c -o Pi-par-V6

!./Pi-par-V6

clang-13: warning: Unknown CUDA version. cuda.h: CUDA_VERSION=11080. Assuming the latest supported version 10.1 [-Wunknown-cuda-version]
pi = 3.141593, 100000000 steps, 1.714305 secs
 